# 第7回講義 演習

## 課題1. 畳み込みニューラルネットワーク(Convolutional Neural Networks)の実装と学習

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data

rng = np.random.RandomState(1234)
random_state = 42

### 1. MNISTデータセットの読み込み

畳み込みニューラルネットワーク(CNN)ではMNISTを4次元テンソルとして扱います. MNISTは縦と横のサイズが28×28で, チャネル数は1となるので, 画像サイズは(バッチサイズ, 28,28, 1)となります.

In [ ]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
mnist_X, mnist_y = mnist.train.images, mnist.train.labels
mnist_X = mnist_X.reshape((mnist_X.shape[0], 28, 28, 1))

train_X, valid_X, train_y, valid_y = train_test_split(mnist_X, mnist_y, test_size=0.1, random_state=42)

### 2. 畳み込みとプーリング in tensorflow

#### 畳み込み: tf.nn.conv2d [\[link\]](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)

- 入力または隠れ層$X_{i, j}^{k}$
    - 次元数4$(n,i,j,k)$
        - $n$：バッチサイズ
        - $i$：入力の行数
        - $j$：入力の列数
        - $k$：入力のチャネル数
- 畳み込みのフィルタ（重み）$W_{k,l}^{i,j}$
    - 次元数4$(i,j,k,l)$
        - $i$: フィルタの行数
        - $j$: フィルタの列数
        - $k$: 入力のチャネル数
        - $l$: 出力のチャネル数(フィルタ数)
    - ストライド：フィルタを適用する位置の間隔
    - ゼロパディング：入力の周りに値0の縁を加えます
        - 入力のサイズを保つ為，フィルタの縦or横の次元が$F$のときパディング数を$(F-1)/2$とします.
- フィルタ後のサイズは，入力の縦or横の次元数$N$，フィルタの縦or横の次元数$F$，ストライドの縦or横の量$S$で決まります.
    - $(N-F)/S+1$

sample image & filter

In [ ]:
# 入力 (4次元)
x = tf.placeholder(tf.float32)

# サンプル画像
sample_image = np.array([[1, 1, 1, 0, 0],
                         [0, 1, 1, 1, 0],
                         [0, 0, 1, 1, 1],
                         [0, 0, 1, 1, 0],
                         [0, 1, 1, 0, 0]]
                       ).astype('float32').reshape(1, 5, 5, 1)

# フィルタ
W = np.array([[1, 0, 1],
              [0, 1, 0],
              [1, 0, 1]]).astype('float32').reshape(3, 3, 1, 1)

- ストライド: (1, 1)
- パディング: なし ('VALID')
- 出力のサイズ: (5-3)/1+1=3

In [ ]:
convoluted_image = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='VALID')

with tf.Session() as sess:
    print(sess.run(convoluted_image, feed_dict={x: sample_image}).reshape(3, 3))

- ストライド: (2, 2)
- パディング: なし
- 出力のサイズ: (5-3)/2+1=2

In [ ]:
convoluted_image = tf.nn.conv2d(x, W, strides=[1,2,2,1], padding='VALID')

with tf.Session() as sess:
    print(sess.run(convoluted_image, feed_dict={x: sample_image}).reshape(2, 2))

- ストライド: (1, 1)
- パディング: (1, 1) 出力サイズが入力と同じになるように
- 出力のサイズ: (5-3+2)/1+1=5

In [ ]:
convoluted_image = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

with tf.Session() as sess:
    print(sess.run(convoluted_image, feed_dict={x: sample_image}).reshape(5, 5))

#### プーリング: tf.nn.max_pool \[[link\]](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool), tf.nn.avg_pool \[[link\]](https://www.tensorflow.org/api_docs/python/tf/nn/avg_pool), etc.

- プーリングには次の種類があります.
    - Max pooling
    - Sum pooling
    - Mean pooling
    - その他Lpプーリングなど
- Convと同様, ストライドやパディングも考えることがあります.

Sample image

In [ ]:
# 入力
x = tf.placeholder(tf.float32)

sample_image = np.array([[77, 80, 82, 78, 70],
                         [83, 78, 80, 83, 82],
                         [87, 82, 81, 80, 74],
                         [87, 87, 85, 77, 66],
                         [84, 79, 77, 78, 76]]
                        ).astype("float32").reshape(1, 5, 5, 1)

- ウィンドウサイズ: (2, 2)
- ストライド: (2, 2)
- プーリング: max

In [ ]:
pooled_image = tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

with tf.Session() as sess:
    print(sess.run(pooled_image, feed_dict={x: sample_image}).reshape(2, 2))

- ウィンドウサイズ: (2, 2)
- ストライド: (1, 1)
- プーリング: max

In [ ]:
pooled_image = tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,1,1,1], padding='VALID')

with tf.Session() as sess:
    print(sess.run(pooled_image, feed_dict={x: sample_image}).reshape(4, 4))

- ウィンドウサイズ: (2, 2)
- ストライド: (2, 2)
- プーリング: mean

In [ ]:
pooled_image = tf.nn.avg_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

with tf.Session() as sess:
    print(sess.run(pooled_image, feed_dict={x: sample_image}).reshape(2, 2))

### 3. 各層クラスの実装

#### 畳み込み層

In [ ]:
class Conv:
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
        # Xavier Initialization
        fan_in = np.prod(filter_shape[:3])
        fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
        self.W = tf.Variable(rng.uniform(
                        low=-np.sqrt(6/(fan_in + fan_out)),
                        high=np.sqrt(6/(fan_in + fan_out)),
                        size=filter_shape
                    ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b') # バイアスはフィルタごとなので, 出力フィルタ数と同じ次元数
        self.function = function
        self.strides = strides
        self.padding = padding

    def f_prop(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)

#### プーリング層

In [ ]:
class Pooling:
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
    
    def f_prop(self, x):
        return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)

#### 平滑化層（4次元->2次元）

In [ ]:
class Flatten:
    def f_prop(self, x):
        return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))

#### 全結合層

In [ ]:
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        # Xavier Initialization
        self.W = tf.Variable(rng.uniform(
                        low=-np.sqrt(6/(in_dim + out_dim)),
                        high=np.sqrt(6/(in_dim + out_dim)),
                        size=(in_dim, out_dim)
                    ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function

    def f_prop(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)

### 4. 計算グラフ構築 & パラメータの更新設定

今Chapterから `tf` の `Optimizer` が使用可能になります.

使い方としては, 最小化したい `cost` に対して以下のように学習率を指定することで
```
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
```
勾配降下法によるパラメータ更新のオペレーションを作成することができます. 勾配降下法以外にもAdagrad, Adam等色々あるので, 詳しくは公式のドキュメント[[link]](https://www.tensorflow.org/api_guides/python/train)を参照してください.

In [ ]:
layers = [                            # (縦の次元数)x(横の次元数)x(チャネル数)
    Conv((5, 5, 1, 20), tf.nn.relu),  # 28x28x 1 -> 24x24x20
    Pooling((1, 2, 2, 1)),            # 24x24x20 -> 12x12x20
    Conv((5, 5, 20, 50), tf.nn.relu), # 12x12x20 ->  8x 8x50
    Pooling((1, 2, 2, 1)),            #  8x 8x50 ->  4x 4x50
    Flatten(),
    Dense(4*4*50, 10, tf.nn.softmax)
]

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])

def f_props(layers, x):
    for layer in layers:
        x = layer.f_prop(x)
    return x

y = f_props(layers, x)

cost = -tf.reduce_mean(tf.reduce_sum(t * tf.log(tf.clip_by_value(y, 1e-10, 1.0)), axis=1)) # tf.log(0)によるnanを防ぐ
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

valid = tf.argmax(y, 1)

### 5. 学習

In [ ]:
n_epochs = 10
batch_size = 100
n_batches = train_X.shape[0]//batch_size

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        train_X, train_y = shuffle(train_X, train_y, random_state=random_state)
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            sess.run(train, feed_dict={x: train_X[start:end], t: train_y[start:end]})
        pred_y, valid_cost = sess.run([valid, cost], feed_dict={x: valid_X, t: valid_y})
        print('EPOCH:: %i, Validation cost: %.3f, Validation F1: %.3f' % (epoch + 1, valid_cost, f1_score(np.argmax(valid_y, 1).astype('int32'), pred_y, average='macro')))